In [1]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "8f58df9a66485e9ea9149b8b599cb14eb71832dc"

# Login to Weights & Biases
wandb.login(key=api_key)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)
import base64
import io  # For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

env = gym.make('CartPole-v1')
env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

wandb.init(project="cartpole-reinforce", config={  # Initialize wandb first
    "hidden_size": 32,  # Default values
    "n_episodes": 1000,
    "max_t": 1000,
    "gamma": 0.99,
    "print_every": 100,
    "lr": 1e-2,
})

config = {
    "hidden_size": wandb.config.hidden_size,  # Access wandb.config after initialization
    "n_episodes": wandb.config.n_episodes,
    "max_t": wandb.config.max_t,
    "gamma": wandb.config.gamma,
    "print_every": wandb.config.print_every,
    "lr": wandb.config.lr,
}

class Policy(nn.Module):
    def __init__(self, state_size=4, action_size=2, hidden_size=config["hidden_size"]):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)

def reinforce(policy, optimizer, n_episodes=2000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for e in range(1, n_episodes):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Collect trajectory
        for t in range(max_t):
            # Sample the action from current policy
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        # Calculate total expected reward
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # Recalculate the total reward applying discounted factor
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a,b in zip(discounts, rewards)])

        # Calculate the loss
        policy_loss = []
        for log_prob in saved_log_probs:
            # Note that we are using Gradient Ascent, not Descent. So we need to calculate it with negative rewards.
            policy_loss.append(-log_prob * R)
        # After that, we concatenate whole policy loss in 0th dimension
        policy_loss = torch.cat(policy_loss).sum()

        # Backpropagation
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
        if np.mean(scores_deque) >= 195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e - 100, np.mean(scores_deque)))
            break
    return scores

############################################For Regret minimization###################################

# def reinforce(policy, optimizer, n_episodes=config["n_episodes"], max_t=config["max_t"], gamma=config["gamma"], print_every=config["print_every"]):
#     scores_deque = deque(maxlen=100)
#     scores = []
#     for e in range(1, n_episodes):
#         # ... (the rest of the code remains the same)
#         if e % print_every == 0:
#             print('Episode {}\\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
#             regret = 500 - np.mean(scores_deque)  # Calculate regret
#             wandb.log({"Regret": regret})
#         if np.mean(scores_deque) >= 495.0:  # Adjust the threshold for early stopping
#             print('Environment solved in {:d} episodes!\\tAverage Score: {:.2f}'.format(e - 100, np.mean(scores_deque)))
#             break
#     wandb.run.summary["final_regret"] = 500 - np.mean(scores_deque)  # Log final regret
#     return scores




#############################################################################

wandb.init(project="cartpole-reinforce", config=config)  # Initialize wandb

policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=config["lr"])
scores = reinforce(policy, optimizer)
wandb.finish()  # Finish wandb run

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
wandb: Currently logged in as: bhavik-160990105023. Use `wandb login --relogin` to force relogin


cpu
observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: Discrete(2)


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 100	Average Score: 40.21
Episode 200	Average Score: 97.11
Episode 300	Average Score: 44.21
Episode 400	Average Score: 40.40
Episode 500	Average Score: 39.88
Episode 600	Average Score: 46.83
Episode 700	Average Score: 69.05
Episode 800	Average Score: 86.48
Episode 900	Average Score: 118.85
Episode 1000	Average Score: 174.49
Environment solved in 908 episodes!	Average Score: 195.61


In [3]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Average Reward',
      'goal': 'maximize'
    },
    'parameters': {
        'hidden_size': {
            'values': [32,64,128]
        },
        'max_t': {
            'values': [500, 1000, 1500]
        },
        'lr': {
            'values': [1e-2, 1e-3,1e-4]
        }
    }
}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# import wandb

# def train(config=None):
#     with wandb.init(config=config):
#         config = wandb.config

#         # Define the hyperparameters
#         hidden_size = config.hidden_size
#         n_episodes = config.n_episodes
#         max_t = config.max_t
#         lr = config.lr

#         # Initialize the policy and optimizer
#         policy = Policy(hidden_size=hidden_size).to(device)
#         optimizer = optim.Adam(policy.parameters(), lr=lr)

#         def reinforce(policy, optimizer, n_episodes=n_episodes, max_t=max_t, gamma=1.0, print_every=100):
#             scores_deque = deque(maxlen=100)
#             scores = []

#             for e in range(1, n_episodes+1):
#                 saved_log_probs = []
#                 rewards = []
#                 state = env.reset()

#                 # Collect trajectory
#                 for t in range(max_t):
#                     # Sample the action from current policy
#                     action, log_prob = policy.act(state)
#                     saved_log_probs.append(log_prob)
#                     state, reward, done, _ = env.step(action)
#                     rewards.append(reward)
#                     if done:
#                         break

#                 # Calculate the total regret
#                 scores_deque.append(sum(rewards))
#                 scores.append(sum(rewards))

#                 # Recalculate the total regret applying discounted factor
#                 discounts = [gamma ** i for i in range(len(rewards) + 1)]
#                 R = sum([a * b for a, b in zip(discounts, rewards)])

#                 # Calculate the loss
#                 policy_loss = []
#                 for log_prob in saved_log_probs:
#                     # Minimizing the regret instead of maximizing the reward
#                     policy_loss.append(log_prob * R)
#                 policy_loss = torch.cat(policy_loss).sum()

#                 # Backpropagation
#                 optimizer.zero_grad()
#                 policy_loss.backward()
#                 optimizer.step()

#                 if e % print_every == 0:
#                     print(f'Episode {e}\tAverage Regret: {np.mean(scores_deque):.2f}')

#                 wandb.log({"regret": np.mean(scores_deque)})

#             return scores

#         scores = reinforce(policy, optimizer)
#         wandb.run.summary["best_regret"] = np.min(scores)

# # Run the sweep
# sweep_id = wandb.sweep(sweep_config, entity="bhavik-160990105023", project="assignment2")
# wandb.agent(sweep_id, train, count=20)

In [5]:
# sweep_config = {
#     'method': 'random',
#     'metric': {
#       'name': 'final_regret',
#       'goal': 'minimize'
#     },
#     'parameters': {
#         'hidden_size':{
#             'values': [1000, 2000, 3000]
#         },
#         'n_episodes': {
#             'values': [0, 0.0005, 0.005]
#         },
#         'max_t': {
#             'values': [500, 1000, 1500]
#         },
#         'lr': {
#             'values': [1e-2, 1e-3, 1e-4]
#         }
#     }
# }

In [6]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity="bhavik-160990105023", project="assignment2")

Create sweep with ID: ilvbh3p3
Sweep URL: https://wandb.ai/bhavik-160990105023/assignment2/sweeps/ilvbh3p3


In [7]:
def train(config=None):
    # Set default values for hyperparameters
    default_config = {
        "hidden_size": 32,
        "n_episodes": 1000,
        "max_t": 1000,
        "gamma": 1.0,
        "print_every": 100,
        "lr": 1e-2,
    }

    # Initialize a new wandb run
    run = wandb.init(config=config, reinit=True)

    # If config is None, use the default values
    if config is None:
        run.config.update(default_config, allow_val_change=True)
        config = run.config

    # Initialize the environment and seed
    env = gym.make('CartPole-v1')
    env.seed(0)

    # Create the policy network with the specified hyperparameters
    policy = Policy(state_size=4, action_size=2, hidden_size=config.hidden_size).to(device)
    optimizer = optim.Adam(policy.parameters(), lr=config.lr)

    # Run the REINFORCE algorithm with the specified hyperparameters
    scores = reinforce(policy, optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

    # Log the final score as a summary metric
    run.summary["final_score"] = np.mean(scores[-100:])

    # Finish the wandb run
    run.finish()

    # Return any necessary values or metrics
    return np.mean(scores[-100:])

In [8]:
sweep_id = wandb.sweep(sweep_config, project="cartpole-reinforce WB")
wandb.agent(sweep_id, train, count=50)

Create sweep with ID: e5bfvrli
Sweep URL: https://wandb.ai/bhavik-160990105023/cartpole-reinforce%20WB/sweeps/e5bfvrli


wandb: Agent Starting Run: ai7l0bkq with config:
wandb: 	hidden_size: 32
wandb: 	lr: 0.01
wandb: 	max_t: 500


wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_t' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Pl

Episode 100	Average Score: 33.13
Episode 200	Average Score: 34.83
Episode 300	Average Score: 59.76
Episode 400	Average Score: 39.80
Episode 500	Average Score: 57.38
Episode 600	Average Score: 60.30
Episode 700	Average Score: 64.93
Episode 800	Average Score: 52.28
Episode 900	Average Score: 41.01


final_score,54.53


...

wandb: Agent Starting Run: p4n634y7 with config:
wandb: 	hidden_size: 128
wandb: 	lr: 0.0001
wandb: 	max_t: 1500


wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_t' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).


Episode 100	Average Score: 26.79
Episode 200	Average Score: 29.19
Episode 300	Average Score: 25.27
Episode 400	Average Score: 26.25
Episode 500	Average Score: 27.32
Episode 600	Average Score: 29.61
Episode 700	Average Score: 29.04
Episode 800	Average Score: 33.03
Episode 900	Average Score: 33.62


final_score,38.03


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [9]:
# wandb agent <sweep_id>